## Mapa da Área

In [ ]:
import osmnx as ox
import folium
from folium.plugins import HeatMap
import geopandas
import numpy as np

In [ ]:
G = ox.graph_from_place('Unicamp', network_type='all')
ox.plot_graph(G)

In [ ]:
ox.save_graph_shapefile(G, '../files/unicamp_wgs84')

In [ ]:
gdf = geopandas.read_file('../files/unicamp20')
data = np.array([[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry])

In [ ]:
m = folium.Map(location=data.mean(axis=0), zoom_start=15)
HeatMap(data).add_to(m)
m

## API Maps

In [ ]:
import requests
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
key = os.getenv('MAPS_KEY')
lat, lon = -22.819034, -47.071818

In [ ]:
metadata = requests.get(
    'https://maps.googleapis.com/maps/api/streetview/metadata',
    params={
        'location': f'{lat},{lon}',
        'key': key
    }
)

In [ ]:
metadata_json = metadata.json()

In [ ]:
metadata_json

In [ ]:
heading = 0
pitch = 0
pano_id = metadata_json['pano_id']
image_url = f'http://maps.googleapis.com/maps/api/streetview?size=400x400&pano={pano_id}&fov=60&heading={heading}&pitch={pitch}&sensor=false&key={key}'

In [ ]:
image_data = requests.get(
    'http://maps.googleapis.com/maps/api/streetview',
    params={
        'size': '400x400',
        'pano': pano_id,
        'fov': 60,
        'heading': heading,
        'pitch': pitch,
        'sensor': 'false',
        'key': key
    },
    stream=True
)

In [ ]:
img = np.array(Image.open(image_data.raw))

In [ ]:
plt.imshow(img)

## Green Index

In [ ]:
import numpy as np
import os, os.path
from PIL import Image
import glob
import matplotlib.pyplot as plt

In [ ]:
def graythresh(array,level):
    '''array: is the numpy array waiting for processing
    return thresh: is the result got by OTSU algorithm
    if the threshold is less than level, then set the level as the threshold
    by Xiaojiang Li
    '''

    maxVal = np.max(array)
    minVal = np.min(array)

    # if the inputImage is a float of double dataset then we transform the data
    # in to byte and range from [0 255]
    if maxVal <= 1:
        array = array*255
    elif maxVal >= 256:
        array = np.int((array - minVal)/(maxVal - minVal))

    # turn the negative to natural number
    negIdx = np.where(array < 0)
    array[negIdx] = 0

    # calculate the hist of 'array'
    dims = np.shape(array)
    hist = np.histogram(array,range(257))
    P_hist = hist[0]*1.0/np.sum(hist[0])

    omega = P_hist.cumsum()

    temp = np.arange(256)
    mu = P_hist*(temp+1)
    mu = mu.cumsum()

    n = len(mu)
    mu_t = mu[n-1]

    sigma_b_squared = (mu_t*omega - mu)**2/(omega*(1-omega))

    # try to found if all sigma_b squrered are NaN or Infinity
    indInf = np.where(sigma_b_squared == np.inf)

    CIN = 0
    if len(indInf[0])>0:
        CIN = len(indInf[0])

    maxval = np.max(sigma_b_squared)

    IsAllInf = CIN == 256
    if IsAllInf !=1:
        index = np.where(sigma_b_squared==maxval)
        idx = np.mean(index)
        threshold = (idx - 1)/255.0
    else:
        threshold = level

    if np.isnan(threshold):
        threshold = level

    return threshold

In [ ]:
panoidlist = []
gsvimgs = glob.glob('../files/gvi-unicamp/images/*.png')
gsvimgs = np.random.choice(gsvimgs, 10)

for img_file in gsvimgs:

    im = np.array(Image.open(img_file))

    I = im/255.0

    red = I[:,:,0]
    green = I[:,:,1]
    blue = I[:,:,2]

    # calculate the difference between green band with other two bands
    green_red_Diff = green - red
    green_blue_Diff = green - blue

    ExG = green_red_Diff + green_blue_Diff
    diffImg = green_red_Diff * green_blue_Diff

    redThreImgU = red < 0.6
    greenThreImgU = green < 0.9
    blueThreImgU = blue < 0.6

    shadowRedU = red < 0.3
    shadowGreenU = green < 0.3
    shadowBlueU = blue < 0.3
    del red, blue, green, I

    greenImg1 = redThreImgU * blueThreImgU*greenThreImgU
    greenImgShadow1 = shadowRedU*shadowGreenU*shadowBlueU

    greenImg3 = diffImg > 0.0
    greenImg4 = green_red_Diff > 0
    threshold = graythresh(ExG, 0.1)

    if threshold > 0.1:
        threshold = 0.1
    elif threshold < 0.05:
        threshold = 0.05

    greenImg2 = ExG > threshold
    greenImgShadow2 = ExG > 0.05
    greenImg = greenImg1*greenImg2 + greenImgShadow2*greenImgShadow1

    greenPxlNum = np.sum(greenImg)
    greenPercent = greenPxlNum / greenImg.size * 100

    f = plt.figure(figsize=(10,10))
    f.add_subplot(1,3,1)
    plt.imshow(im)
    plt.title('Original Image')

    f.add_subplot(1,3,2)
    plt.imshow(ExG)
    plt.title('Image Green Pixels')

    f.add_subplot(1,3,3)
    plt.imshow(greenImg)
    plt.title('Image Vegetation')

    plt.suptitle(f'The green view index is {greenPercent:.2f}', y=0.7)
    plt.tight_layout()
    plt.show()